# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,281.76,85,86,0.28,CL,1690423527
1,1,port-aux-francais,-49.3500,70.2167,277.59,78,90,16.12,TF,1690423526
2,2,jackson,42.4165,-122.8345,305.08,22,0,7.20,US,1690423608
3,3,ushuaia,-54.8000,-68.3000,272.96,69,0,2.57,AR,1690423657
4,4,longyearbyen,78.2186,15.6401,284.06,71,75,1.03,SJ,1690423533


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
myplot = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles = "OSM", frame_width= 700, frame_height =500,
                                    size="Humidity", scale = 0.5, color = "City")
# Display the map
# YOUR CODE HERE

myplot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
favcities = city_data_df[city_data_df["Country"]== "US"]
favcities = favcities[(favcities["Cloudiness"] >= 25) & (favcities["Cloudiness"] <= 75)]
# Drop any rows with null values
# YOUR CODE HERE
favcities.dropna()
# Display sample data
# YOUR CODE HERE
favcities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,41,san patricio,28.0170,-97.5169,303.25,72,53,5.74,US,1690423663
55,55,wailua homesteads,22.0669,-159.3780,302.39,78,75,7.20,US,1690423666
56,56,hilo,19.7297,-155.0900,301.00,81,75,4.12,US,1690423607
134,134,palmer,42.1584,-72.3287,297.38,72,40,4.63,US,1690423603
135,135,college,64.8569,-147.8028,302.92,52,75,1.34,US,1690423680
150,150,lihue,21.9789,-159.3672,302.73,73,75,7.20,US,1690423532
196,196,ketchikan,55.3422,-131.6461,292.41,68,75,4.12,US,1690423636
210,210,kailua-kona,19.6406,-155.9956,302.32,80,40,5.66,US,1690423699
218,218,fallon,39.4735,-118.7774,307.29,15,40,6.17,US,1690423446
267,267,kapa'a,22.0752,-159.3190,303.14,78,75,7.20,US,1690423716


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = favcities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
41,san patricio,US,28.0170,-97.5169,72,
55,wailua homesteads,US,22.0669,-159.3780,78,
56,hilo,US,19.7297,-155.0900,81,
134,palmer,US,42.1584,-72.3287,72,
135,college,US,64.8569,-147.8028,52,
150,lihue,US,21.9789,-159.3672,73,
196,ketchikan,US,55.3422,-131.6461,68,
210,kailua-kona,US,19.6406,-155.9956,80,
218,fallon,US,39.4735,-118.7774,15,
267,kapa'a,US,22.0752,-159.3190,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
category = "accommodation.hotel"
params = {
    "apiKey":geoapify_key,
    "categories":category
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    mylat = row["Lat"]
    mylon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{mylon},{mylat},{radius}"# YOUR CODE HERE
    params["bias"] = f"proximity:{mylon},{mylat}" # YOUR CODE HERE
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    print(name_address)
    # Convert the API response to JSON format
    name_address = name_address.json()# YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
<Response [200]>
san patricio - nearest hotel: No hotel found
<Response [200]>
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
<Response [200]>
hilo - nearest hotel: Dolphin Bay Hotel
<Response [200]>
palmer - nearest hotel: No hotel found
<Response [200]>
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
<Response [200]>
lihue - nearest hotel: Kauai Palms
<Response [200]>
ketchikan - nearest hotel: Cape Fox Lodge
<Response [200]>
kailua-kona - nearest hotel: Kona Seaside Hotel
<Response [200]>
fallon - nearest hotel: Value Inn
<Response [200]>
kapa'a - nearest hotel: Pono Kai Resort
<Response [200]>
hawaiian paradise park - nearest hotel: No hotel found
<Response [200]>
juneau - nearest hotel: Ramada by Wyndham Juneau
<Response [200]>
badger - nearest hotel: Hotel North Pole


,City,Country,Lat,Lng,Humidity,Hotel Name
41,san patricio,US,28.0170,-97.5169,72,No hotel found
55,wailua homesteads,US,22.0669,-159.3780,78,Hilton Garden Inn Kauai Wailua Bay
56,hilo,US,19.7297,-155.0900,81,Dolphin Bay Hotel
134,palmer,US,42.1584,-72.3287,72,No hotel found
135,college,US,64.8569,-147.8028,52,BEST WESTERN PLUS Pioneer Park Inn
150,lihue,US,21.9789,-159.3672,73,Kauai Palms
196,ketchikan,US,55.3422,-131.6461,68,Cape Fox Lodge
210,kailua-kona,US,19.6406,-155.9956,80,Kona Seaside Hotel
218,fallon,US,39.4735,-118.7774,15,Value Inn
267,kapa'a,US,22.0752,-159.3190,78,Pono Kai Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
myhotelplot = hotel_df.hvplot.points("Lng", "Lat", geo=True, tiles = "OSM", frame_width= 700, frame_height =500,
                                    size="Humidity", scale = 0.5, color = "City",
                                    hover_cols= ["Hotel Name", "Country"])
# Display the map
# YOUR CODE HERE
myhotelplot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [ ]:
#Updated 7/29/23